In [1]:
import glob
import random
import os
import numpy as np

from matplotlib.pyplot import imshow
from torch.utils.data import Dataset
from PIL import Image
import PIL
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import json
from os import path, walk, makedirs
from sys import exit, stderr

from cv2 import fillPoly, imwrite
from shapely import wkt
from shapely.geometry import mapping, Polygon, box
from skimage.io import imread
from tqdm import tqdm
import imantics 

In [2]:
data_folder = '/home/shuza/Code/xview2-baseline/data'

masks_path = '/home/shuza/Code/xview2-baseline/data/guatemala-volcano/masks'
imgs_path = '/home/shuza/Code/xview2-baseline/data/guatemala-volcano/images'

ind_mask = '/home/shuza/Code/xview2-baseline/data/guatemala-volcano/masks/guatemala-volcano_00000000_pre_disaster.png'
ind_pre = '/home/shuza/Code/xview2-baseline/data/guatemala-volcano/images/guatemala-volcano_00000000_pre_disaster.png'
ind_post = '/home/shuza/Code/xview2-baseline/data/guatemala-volcano/images/guatemala-volcano_00000000_post_disaster.png'

# Goals

load an image, load the corresponding mask, mask out the image.

- *Semantic map*
- Train GAN on mask -> image pre
- *Image Repair*
- Train GAN on masked image -> image pre

In [ ]:
img_size = 1024

In [ ]:
test_img = Image.open(ind_pre)
test_mask = Image.open(ind_mask)

In [ ]:
imshow(test_img)

In [ ]:
imshow(test_mask)

In [ ]:
mask_arr = np.asarray(test_mask)
img_arr = np.asarray(test_img)

In [ ]:
print(mask_arr.shape)
print(img_arr.shape)
print(test_img.mode)
print(test_mask.mode)
print(test_img.format)
print(test_img.size)
print(test_img.palette)

In [ ]:
# test_img.thumbnail((400,400))

In [ ]:
# test_img.show()

# Composite

In [ ]:
img = test_img.convert('L')
mask = test_mask.convert('L')

In [ ]:
PIL.Image.composite(mask,img,mask)

# Paste one image onto another

In [ ]:
copy_img = test_img.copy()
# position = ((image_copy.width - logo.width), (image_copy.height - logo.height))
copy_img.paste(test_mask,(1024,1024),test_mask)
imshow(copy_img)

# Rotate images

In [ ]:
image_90 = test_img.rotate(90)
imshow(image_90)

# Flip images

In [ ]:
image_flip = test_img.transpose(PIL.Image.FLIP_LEFT_RIGHT)
imshow(image_flip)

# Cropping

In [ ]:
box = (150,200,600,600)
cropped_image = test_img.crop(box)
imshow(cropped_image)

# Auto mask image from data

## Create dataset for image repair

Goes from masked image -> repairs buildings

In [ ]:
masks_path
imgs_path

In [ ]:
img_list = os.listdir(imgs_path)
mask_list = os.listdir(masks_path)
mask_dict = {k:v for v,k in enumerate(mask_list)}
img_dict = {k:v for v,k in enumerate(img_list)}

In [ ]:
input_imgs = []
src_imgs = []

In [ ]:
test = os.listdir('/home/shuza/Code/xview2-baseline/data/hurricane-florence')
test.index('masks')

In [ ]:
# Takes parent folder with subfolder names images and masks,
# Takes target folder, makes the src and target image folders
# Put images through all 3 rotations and all 4 flipped rotations

def create_dataset(dir_params):
    # Go through images and create a dictionary to index all the files.
#     parent_list = os.listdir(dir_params['src_folder'])
    print(dir_params)
    
    if not path.isdir(dir_params['targets_folder']):
        makedirs(dir_params['targets_folder'])
        
    if not path.isdir(dir_params['target_inputs']):
        makedirs(dir_params['target_inputs'])
    folder = 'mexico-earthquake'
#     for folder in parent_list:
    folder_path = os.path.join(dir_params['src_folder'],folder)
    folder_items = os.listdir(folder_path)
    mask_folder = os.path.join(folder_path,'masks')
    images_folder = os.path.join(folder_path,'images')
    img_list = os.listdir(images_folder)
    mask_list = os.listdir(mask_folder)
    img_dict = {k:v for v,k in enumerate(img_list)}
    for mask_file in mask_list:
#         print('mask_file',mask_file)
#         corrected_mask_file = mask_file.split('pre_disaster')[0] + 'pre_disaster.png'
#         index = img_dict[mask_file]
        mask_path = os.path.join(mask_folder,mask_file)
#         img_path = os.path.join(images_folder,img_list[index])
        mask_img = Image.open(mask_path)
#         pre_img = Image.open(img_path)
#         L_img = pre_img.convert('L')
        L_mask = mask_img.convert('L')
#         masked_img = PIL.Image.composite(L_mask,L_img,L_mask)
#         save_img = os.path.join(dir_params['targets_folder'],mask_file)
#         save_masked = os.path.join(dir_params['target_inputs'],mask_file)
        save_mask = os.path.join(dir_params['mask_folder'],mask_file)
#         masked_img.save(save_masked)
#         L_img.save(save_img)
        L_mask.save(save_mask)

In [ ]:
dir_params = {
    'targets_folder':'/home/shuza/Code/xview2-baseline/GAN_data/targets',
    'target_inputs':'/home/shuza/Code/xview2-baseline/GAN_data/inputs',
    'src_folder':'/home/shuza/Code/xview2-baseline/data',
    'mask_folder':'/home/shuza/Code/xview2-baseline/GAN_data/masks'
}

create_dataset(dir_params)

# Test combining building masks from tier 3

In [ ]:
# Test the concept

file_1 = '/home/shuza/Code/xview2-baseline/data/woolsey-fire/masks/woolsey-fire_00000000_pre_disaster_22b37215-685d-450d-9fc8-b2eab7a5ebc1.png' 
file_2 = '/home/shuza/Code/xview2-baseline/data/woolsey-fire/masks/woolsey-fire_00000000_pre_disaster_40d4fa85-2b2f-488e-b3c2-0fee718cac82.png' 
file_3 = '/home/shuza/Code/xview2-baseline/data/woolsey-fire/masks/woolsey-fire_00000000_pre_disaster_fed80ca6-0dba-4bee-9a7c-67d2f218c204.png' 

img_1 = Image.open(file_1)
img_2 = Image.open(file_2)
img_3 = Image.open(file_3)

print(img_1.size,img_1.format)

In [ ]:
file_1.split('pre_disaster')[0] + 'pre_disaster'

In [ ]:
imshow(img_1)

In [ ]:
composite_img = PIL.Image.composite(img_1.convert('L'),img_2.convert('L'),img_1.convert('L'))
composite_img = PIL.Image.composite(img_3.convert('L'),composite_img,img_3.convert('L'))
# Image.alpha_composite(img_2.convert('RGBA'),img_3.convert('RGBA'))
                                                       

In [ ]:
imshow(composite_img)

In [ ]:
print(composite_img.size)
print(img_1.format)

In [ ]:

copy_img = img_1.copy()
# position = ((image_copy.width - logo.width), (image_copy.height - logo.height))
copy_img.paste(img_1,(1024,1024),img_2)
imshow(copy_img)

# Combine all the individual building masks in the tier 3 data

In [ ]:
folder_list = ['/home/shuza/Code/xview2-baseline/data/joplin-tornado',
 '/home/shuza/Code/xview2-baseline/data/lower-puna-volcano' ,
    '/home/shuza/Code/xview2-baseline/data/moore-tornado' ,
'/home/shuza/Code/xview2-baseline/data/nepal-flooding' ,
'/home/shuza/Code/xview2-baseline/data/pinery-bushfire' ,
'/home/shuza/Code/xview2-baseline/data/portugal-wildfire',
 '/home/shuza/Code/xview2-baseline/data/sunda-tsunami' ,
'/home/shuza/Code/xview2-baseline/data/tuscaloosa-tornado', 
'/home/shuza/Code/xview2-baseline/data/woolsey-fire']

test_folder = '/home/shuza/Code/xview2-baseline/GAN_data/combined' 

In [ ]:
print(folder_list)

In [ ]:
# Make progress bar for this.
for folder in folder_list:
    # Gather all the distinct names and all the single masks per name
    mask_folder = os.path.join(folder,'masks')
    print('current folder',mask_folder)
    names = set()
    duplicate_dict = {}
    for mask_file in os.listdir(mask_folder):
#         print(mask_file)
        mask_path = os.path.join(mask_folder,mask_file)
#         print(mask_path)
        generic_name = mask_file.split('pre_disaster')[0] + 'pre_disaster'
#         print('generic_name',generic_name)
        names.add(generic_name)
        if generic_name not in duplicate_dict:
            duplicate_dict[generic_name] = [mask_path]
        else:
            duplicate_dict[generic_name].append(mask_path)
    # Combine all the single building masks into one
    for name in names:
        mask_list = duplicate_dict[name]
        if len(mask_list) > 1:
            src_img = Image.open(mask_list[0]).convert('L')
            for i in range(len(mask_list) - 1):
                dup_img = Image.open(mask_list[i+1]).convert('L')
                src_img = PIL.Image.composite(src_img,dup_img,src_img)
            # save composite
            src_img.save(test_folder + '/' + name + '.png')
        else:
            # don't change anything
            src_img = Image.open(mask_list[0]).convert('L')
            src_img.save(test_folder + '/' + name + '.png')

# Train GAN on data

Two GAN types:
- Image repair - Context encoder
- Generating img from a mask - Pix2pix

## Context Encoder

In [12]:
import torch.nn as nn
import torch.nn.functional as F
import torch

class Generator(nn.Module):
    def __init__(self, channels=3):
        super(Generator, self).__init__()

        def downsample(in_feat, out_feat, normalize=True):
            layers = [nn.Conv2d(in_feat, out_feat, 4, stride=2, padding=1)]
            if normalize:
                layers.append(nn.BatchNorm2d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2))
            return layers

        def upsample(in_feat, out_feat, normalize=True):
            layers = [nn.ConvTranspose2d(in_feat, out_feat, 4, stride=2, padding=1)]
            if normalize:
                layers.append(nn.BatchNorm2d(out_feat, 0.8))
            layers.append(nn.ReLU())
            return layers

        self.model = nn.Sequential(
            *downsample(channels, 32, normalize=False),
            *downsample(32, 64),
            *downsample(64, 128),
            *downsample(128, 256),
            *downsample(256, 512),
            nn.Conv2d(512, 4000, 1),
            *upsample(4000, 512),
            *upsample(512, 256),
            *upsample(256, 128),
            *upsample(128, 64),
            *upsample(64, 32),
            nn.Conv2d(32, channels, 3, 1, 1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)


class Discriminator(nn.Module):
    def __init__(self, channels=3):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, stride, normalize):
            """Returns layers of each discriminator block"""
            layers = [nn.Conv2d(in_filters, out_filters, 3, stride, 1)]
            if normalize:
                layers.append(nn.InstanceNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        layers = []
        in_filters = channels
        for out_filters, stride, normalize in [(64, 2, False), (128, 2, True), (256, 2, True), (512, 2, True),(1024, 1, True)]:
            layers.extend(discriminator_block(in_filters, out_filters, stride, normalize))
            in_filters = out_filters

        layers.append(nn.Conv2d(out_filters, 1, 3, 1, 1))

        self.model = nn.Sequential(*layers)

    def forward(self, img):
        return self.model(img)

In [4]:
class Parameters(object):
    def __init__(self):
        self.mask_size = 64
        self.img_size = 1024
        self.channels = 1
        self.lr = 0.0002
        self.b1 = 0.5
        self.b2 = 0.999
        self.batch_size = 8
        self.n_cpu = 1
        self.n_epochs = 1
        self.sample_interval = 500
        self.latent_dim = 100
        self.inputs = '/home/shuza/Code/xview2-baseline/GAN_data/inputs'
        self.targets = '/home/shuza/Code/xview2-baseline/GAN_data/targets'
        self.masks = '/home/shuza/Code/xview2-baseline/GAN_data/masks'
        
opt = Parameters()

# Test networks

I need to make the masks boolean masks. So set all values > 0 to 1 and else 0.

In [ ]:
test_img_path = '/home/shuza/Code/xview2-baseline/GAN_data/inputs/mexico-earthquake_00000000_pre_disaster.png'
test_img = Image.open(test_img_path)

In [ ]:
# Figure out what patch does
patch_h, patch_w = int(opt.mask_size / 2 ** 3), int(opt.mask_size / 2 ** 3)
patch = (1, patch_h, patch_w)


In [ ]:
print(patch,patch_h,patch_w)

In [ ]:
Tensor =torch.FloatTensor

# Calculate output of image discriminator (PatchGAN)
patch_h, patch_w = int(opt.mask_size / 2 ** 3), int(opt.mask_size / 2 ** 3)
patch = (1, patch_h, patch_w)

# Adversarial ground truths
valid = Variable(Tensor(1024, *patch).fill_(1.0), requires_grad=False)
fake = Variable(Tensor(1024, *patch).fill_(0.0), requires_grad=False)

# Configure input
# test_img = Variable(test_img.type(Tensor))
# masked_imgs = Variable(masked_imgs.type(Tensor))

In [ ]:
print(valid.size())
print(valid)

# Test image dataset class

In [5]:
class ImageDataset(Dataset):
    def __init__(self, src_root,tgt_root,mask_root, transforms_=None, img_size=128, mode="train"):
        self.transform = transforms.Compose(transforms_)
        self.img_size = img_size
        self.mode = mode
        self.src_files = sorted(glob.glob("%s/*.png" % src_root))
        self.src_files = self.src_files[:-40] if mode == "train" else self.src_files[-40:]
        self.tgt_files = sorted(glob.glob("%s/*.png" % tgt_root))
        self.tgt_files = self.tgt_files[:-40] if mode == "train" else self.tgt_files[-40:]
        self.mask_files = sorted(glob.glob("%s/*.png" % mask_root))
        self.mask_files = self.mask_files[:-40] if mode == "train" else self.mask_files[-40:]

    def __getitem__(self, index):

        img = Image.open(self.src_files[index % len(self.src_files)])
        masked_img = Image.open(self.tgt_files[index % len(self.tgt_files)])
        mask_img = Image.open(self.mask_files[index % len(self.mask_files)])
#         print(type(img),type(masked_img))
        img = self.transform(img)
        masked_img = self.transform(masked_img)
        mask_img = self.transform(mask_img)

#         print(type(img),type(masked_img))
        return img, masked_img, mask_img

    def __len__(self):
        return len(self.src_files) + len(self.tgt_files) + len(self.mask_files)

In [6]:
transforms_ = [
    transforms.Resize((opt.img_size, opt.img_size), Image.BICUBIC),
    transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
]

dataloader = DataLoader(
    ImageDataset(opt.inputs,opt.targets,opt.masks,transforms_=transforms_,img_size=1024),
    batch_size=opt.batch_size,
    shuffle=True,
    num_workers=opt.n_cpu,
)
test_dataloader = DataLoader(
    ImageDataset(opt.inputs,opt.targets,opt.masks,transforms_=transforms_,img_size=1024,mode="test"),
    batch_size=opt.batch_size,
    shuffle=False,
    num_workers=opt.n_cpu,
)

In [ ]:
# print(len(dataloader))
# print(next(iter(dataloader)))

In [13]:
generator = Generator(channels=opt.channels)
discriminator = Discriminator(channels=opt.channels)

# Test using mask to isolate masked parts

In [8]:
src,tgt,mask = next(iter(dataloader))

In [9]:
masked_parts = generator(src)

In [10]:
print(src.size(),tgt.size(),mask.size(),masked_parts.size())

torch.Size([8, 1, 1024, 1024]) torch.Size([8, 1, 1024, 1024]) torch.Size([8, 1, 1024, 1024]) torch.Size([8, 1, 1024, 1024])


In [14]:
fv = discriminator(masked_parts)

In [15]:
print(fv.size())

torch.Size([8, 1, 64, 64])


In [ ]:
print(src,'\n')
print(mask)

In [ ]:
# times the generated image and src image by the mask and get the pixel loss
tgt_pixels = tgt * mask
gen_pixels = masked_parts * mask
print(tgt_pixels.size(),gen_pixels.size())

In [ ]:
# print(gen_pixels)

In [ ]:
gen_img = gen_pixels.detach()[0,:,:,:].reshape(1024,1024)
tgt_img = tgt_pixels.detach()[0,:,:,:].reshape(1024,1024)

In [ ]:
imshow(tgt_img)

In [ ]:
imshow(tgt[0,:,:,:].reshape(1024,1024))

# Train baby train!

In [16]:
def save_sample(batches_done):
    masked_samples,samples,masks  = next(iter(test_dataloader))
    samples = Variable(samples.type(Tensor))
    masked_samples = Variable(masked_samples.type(Tensor))
    masks = Variable(masks.type(Tensor))
#     i = i[0].item()  # Upper-left coordinate of mask
#     # Generate inpainted image
    gen_img = generator(masked_samples)
    generated_buildings = gen_img * masks
#     filled_samples = masked_samples.clone()
#     filled_samples[:, :, i : i + opt.mask_size, i : i + opt.mask_size] = gen_mask
#     # Save sample
#     sample = torch.cat((masked_samples.data, filled_samples.data, samples.data), -2)
    save_image(generated_buildings, "test_images/%d.png" % batches_done, nrow=6, normalize=True)

In [17]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

# Loss function
adversarial_loss = torch.nn.MSELoss()
pixelwise_loss = torch.nn.L1Loss()

# Initialize generator and discriminator
generator = Generator(channels=opt.channels)
discriminator = Discriminator(channels=opt.channels)

In [18]:
# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

transforms_ = [
    transforms.Resize((opt.img_size, opt.img_size), Image.BICUBIC),
    transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
]

dataloader = DataLoader(
    ImageDataset(opt.inputs,opt.targets,opt.masks,transforms_=transforms_,img_size=1024),
    batch_size=opt.batch_size,
    shuffle=True,
    num_workers=opt.n_cpu,
)
test_dataloader = DataLoader(
    ImageDataset(opt.inputs,opt.targets,opt.masks,transforms_=transforms_,img_size=1024,mode="test"),
    batch_size=opt.batch_size,
    shuffle=False,
    num_workers=opt.n_cpu,
)

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

Tensor =  torch.FloatTensor

In [19]:
# src,tgt,mask = next(iter(dataloader))
# print(type(src),type(tgt))
# print(src.size(),tgt.size())
# valid = Variable(Tensor(12, 1024,1024).fill_(1.0), requires_grad=False)

In [20]:
"""
Masked parts: src image * mask = only src buildings
masks: black and white mask of buildings and background
masked imgs: src image with buildings masked out
imgs: src image
masked gen: generated image with background masked out
"""

for epoch in range(opt.n_epochs):
    for i, (masked_imgs,imgs, masks) in enumerate(dataloader):

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.shape[0], 1,64,64).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.shape[0], 1,64,64).fill_(0.0), requires_grad=False)
        
        masked_valid = valid * mask
        masked_fake = fake * mask

        buildings = imgs * masks
        # Configure input
        imgs = Variable(imgs.type(Tensor))
        masked_imgs = Variable(masked_imgs.type(Tensor))
        masks = Variable(masks.type(Tensor))
        buildings = Variable(buildings.type(Tensor))
        

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Generate a batch of images
        gen_img = generator(masked_imgs)
        
        generated_buildings = gen_img * masks

        # Adversarial and pixelwise loss
        g_adv = adversarial_loss(discriminator(generated_buildings), masked_valid)
        g_pixel = pixelwise_loss(generated_buildings, buildings)
        # Total loss
        g_loss = 0.001 * g_adv + 0.999 * g_pixel

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(buildings), masked_valid)
        fake_loss = adversarial_loss(discriminator(generated_buildings.detach()), masked_fake)
        d_loss = 0.5 * (real_loss + fake_loss)

        d_loss.backward()
        optimizer_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G adv: %f, pixel: %f]"
            % (epoch, opt.n_epochs, i, len(dataloader), d_loss.item(), g_adv.item(), g_pixel.item())
        )

        # Generate sample at sample interval
        batches_done = epoch * len(dataloader) + i
        if batches_done % opt.sample_interval == 0:
            save_sample(batches_done)

OSError: [Errno 12] Cannot allocate memory

# Generate img from mask

Data already exists in the folders. Just a matter of linking up the src_img and the mask


# Pix2pix

In [ ]:


def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)


##############################
#           U-NET
##############################


class UNetDown(nn.Module):
    def __init__(self, in_size, out_size, normalize=True, dropout=0.0):
        super(UNetDown, self).__init__()
        layers = [nn.Conv2d(in_size, out_size, 4, 2, 1, bias=False)]
        if normalize:
            layers.append(nn.InstanceNorm2d(out_size))
        layers.append(nn.LeakyReLU(0.2))
        if dropout:
            layers.append(nn.Dropout(dropout))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)


class UNetUp(nn.Module):
    def __init__(self, in_size, out_size, dropout=0.0):
        super(UNetUp, self).__init__()
        layers = [
            nn.ConvTranspose2d(in_size, out_size, 4, 2, 1, bias=False),
            nn.InstanceNorm2d(out_size),
            nn.ReLU(inplace=True),
        ]
        if dropout:
            layers.append(nn.Dropout(dropout))

        self.model = nn.Sequential(*layers)

    def forward(self, x, skip_input):
        x = self.model(x)
        x = torch.cat((x, skip_input), 1)

        return x


class GeneratorUNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super(GeneratorUNet, self).__init__()

        self.down1 = UNetDown(in_channels, 64, normalize=False)
        self.down2 = UNetDown(64, 128)
        self.down3 = UNetDown(128, 256)
        self.down4 = UNetDown(256, 512, dropout=0.5)
        self.down5 = UNetDown(512, 512, dropout=0.5)
        self.down6 = UNetDown(512, 512, dropout=0.5)
        self.down7 = UNetDown(512, 512, dropout=0.5)
        self.down8 = UNetDown(512, 512, normalize=False, dropout=0.5)

        self.up1 = UNetUp(512, 512, dropout=0.5)
        self.up2 = UNetUp(1024, 512, dropout=0.5)
        self.up3 = UNetUp(1024, 512, dropout=0.5)
        self.up4 = UNetUp(1024, 512, dropout=0.5)
        self.up5 = UNetUp(1024, 256)
        self.up6 = UNetUp(512, 128)
        self.up7 = UNetUp(256, 64)

        self.final = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.ZeroPad2d((1, 0, 1, 0)),
            nn.Conv2d(128, out_channels, 4, padding=1),
            nn.Tanh(),
        )

    def forward(self, x):
        # U-Net generator with skip connections from encoder to decoder
        d1 = self.down1(x)
        d2 = self.down2(d1)
        d3 = self.down3(d2)
        d4 = self.down4(d3)
        d5 = self.down5(d4)
        d6 = self.down6(d5)
        d7 = self.down7(d6)
        d8 = self.down8(d7)
        u1 = self.up1(d8, d7)
        u2 = self.up2(u1, d6)
        u3 = self.up3(u2, d5)
        u4 = self.up4(u3, d4)
        u5 = self.up5(u4, d3)
        u6 = self.up6(u5, d2)
        u7 = self.up7(u6, d1)

        return self.final(u7)
